In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from skimage.transform import resize
import h5py

In [2]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 42x42
x_train_resized = np.zeros((x_train.shape[0], 42, 42))
for i in range(x_train.shape[0]):
    x_train_resized[i] = resize(x_train[i], (42, 42))

x_test_resized = np.zeros((x_test.shape[0], 42, 42))
for i in range(x_test.shape[0]):
    x_test_resized[i] = resize(x_test[i], (42, 42))

# Reshape to add channel dimension (required for Keras)
x_train_resized = x_train_resized.reshape(-1, 42, 42, 1)
x_test_resized = x_test_resized.reshape(-1, 42, 42, 1)

# Normalize pixel values to range [0, 1]
x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [3]:
model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train_resized, y_train, batch_size=128, epochs=15, validation_data=(x_test_resized, y_test))

# Quantize the weights and biases
model_quantized = Sequential()
for layer in model.layers:
    if isinstance(layer, Dense) or isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        weights_quantized = np.round(weights * 4) / 4  # Quantize weights to 2 bits for fixed point part
        biases_quantized = np.round(biases * 4) / 4     # Quantize biases to 2 bits for fixed point part
        layer.set_weights([weights_quantized, biases_quantized])
    model_quantized.add(layer)

model_quantized.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_quantized.save('quantized_mnist_model.h5')

loss, accuracy = model_quantized.evaluate(x_test_resized, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/15


c:\Users\Rishit Thakkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2234 - loss: 2.2082 - val_accuracy: 0.7660 - val_loss: 1.0341
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7173 - loss: 0.9657 - val_accuracy: 0.8572 - val_loss: 0.5435
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7694 - loss: 0.7236 - val_accuracy: 0.8719 - val_loss: 0.4683
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7853 - loss: 0.6747 - val_accuracy: 0.8757 - val_loss: 0.4409
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7927 - loss: 0.6507 - val_accuracy: 0.8819 - val_loss: 0.4216
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8005 - loss: 0.6286 - val_accuracy: 0.8823 - val_loss: 0.4096
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8045 - loss: 0.6184 - val_accuracy: 0.8868 - val_loss: 0.4007
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8077 - loss: 0.6087 - val_accuracy: 0.8892 - val_

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.8561 - loss: 0.6173
Test Loss: 0.5866955518722534
Test Accuracy: 0.8644999861717224
